<a href="https://colab.research.google.com/github/dhighlord/NNCTR/blob/master/run_multivalue_movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dhighlord/NNCTR.git

Cloning into 'NNCTR'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (72/72), done.


In [0]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from NNCTR.deepctr.models import DeepFM
from NNCTR.deepctr.inputs import SparseFeat,get_fixlen_feature_names

In [3]:
if __name__ == "__main__":

    data = pd.read_csv("NNCTR/examples/movielens_sample.txt")
    sparse_features = ["movie_id", "user_id",
                       "gender", "age", "occupation", "zip"]
    target = ['rating']

    # 1.Label Encoding for sparse features,and do simple Transformation for dense features
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    # 2.count #unique features for each sparse field
    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features]
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns
    fixlen_feature_names = get_fixlen_feature_names(linear_feature_columns + dnn_feature_columns)

    # 3.generate input data for model
    train, test = train_test_split(data, test_size=0.2)
    train_model_input = [train[name].values for name in fixlen_feature_names]
    test_model_input = [test[name].values for name in fixlen_feature_names]
    # 4.Define Model,train,predict and evaluate
    model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
    model.compile("adam", "mse", metrics=['mse'], )

    history = model.fit(train_model_input, train[target].values,
                        batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    print("test MSE", round(mean_squared_error(
        test[target].values, pred_ans), 4))

W0809 09:24:45.636941 139888006326144 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0809 09:24:45.860097 139888006326144 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0809 09:24:46.120093 139888006326144 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:253: calling reduce_sum_v1 (from tensorflow.python.ops.math_op

Train on 128 samples, validate on 32 samples
Epoch 1/10
128/128 - 1s - loss: 14.2736 - mean_squared_error: 14.2736 - val_loss: 14.2693 - val_mean_squared_error: 14.2693
Epoch 2/10
128/128 - 0s - loss: 14.1469 - mean_squared_error: 14.1469 - val_loss: 14.1523 - val_mean_squared_error: 14.1523
Epoch 3/10
128/128 - 0s - loss: 14.0062 - mean_squared_error: 14.0062 - val_loss: 14.0285 - val_mean_squared_error: 14.0285
Epoch 4/10
128/128 - 0s - loss: 13.8566 - mean_squared_error: 13.8566 - val_loss: 13.8972 - val_mean_squared_error: 13.8972
Epoch 5/10
128/128 - 0s - loss: 13.6974 - mean_squared_error: 13.6974 - val_loss: 13.7575 - val_mean_squared_error: 13.7575
Epoch 6/10
128/128 - 0s - loss: 13.5277 - mean_squared_error: 13.5277 - val_loss: 13.6083 - val_mean_squared_error: 13.6083
Epoch 7/10
128/128 - 0s - loss: 13.3467 - mean_squared_error: 13.3467 - val_loss: 13.4492 - val_mean_squared_error: 13.4492
Epoch 8/10
128/128 - 0s - loss: 13.1535 - mean_squared_error: 13.1535 - val_loss: 13.27